In [1]:
import os
import random
from PIL import Image, ImageDraw

In [37]:
def rotate_shape(shape, rotates_num=1):
    '''Функция для поворота массива различной размерности на 90 градусов по часовой стрелке'''
    
    for _ in range(rotates_num % 4):  # 4 поворота — это полный оборот, больше не нужно
        shape = [list(row) for row in zip(*shape[::-1])]
    
    return shape

def print_mas(shape):
    for row in shape:
        print(row)
    print()
    return 

In [36]:
test_shape = [[1,2,3],[4,5,6],[7,8,9]]
shape = test_shape[::-1]
zip_shape = zip(*shape)
print_mas(test_shape)
print_mas(shape)
print_mas(zip_shape)


[1, 2, 3]
[4, 5, 6]
[7, 8, 9]

[7, 8, 9]
[4, 5, 6]
[1, 2, 3]

(7, 4, 1)
(8, 5, 2)
(9, 6, 3)



In [ ]:
test_shape4 = [[1,2,3],[4,5,6],[7,8,9]]
test_shape2 = [[1,2],[3,4]]
test_shape3 = [[1,2],[3,4],[5,6]]

In [39]:
for i in range(5):
    print_mas(rotate_shape(test_shape2, rotates_num=i))

[1, 2]
[3, 4]

[3, 1]
[4, 2]

[4, 3]
[2, 1]

[2, 4]
[1, 3]

[1, 2]
[3, 4]



In [40]:
print_mas([[0] * 9 for _ in range(7)])

[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]



In [53]:
CELL_SIZE = 160
GRID_WIDTH = 9
GRID_HEIGHT = 7
IMG_WIDTH = GRID_WIDTH * CELL_SIZE
IMG_HEIGHT = GRID_HEIGHT * CELL_SIZE


def can_place(grid, shape, x, y):
    for dy, row in enumerate(shape):           # проходим по строкам формы
        for dx, val in enumerate(row):
            print(f'Текущая ячейка: {dy},{dx}')         # и по элементам в строке
            if val == 1:                       # если текущая ячейка непустая (1)
                gx, gy = x + dx, y + dy        # вычисляем координаты на сетке
                if gx >= GRID_WIDTH or gy >= GRID_HEIGHT or grid[gy][gx] == 1:
                    return False               # нельзя ставить: выходит за границы или ячейка занята
    return True

def fill_array(val=0, grid_width = 1, grid_height = 1):
    '''Создает массив заданной размерности заполненный значением val'''
    return [[val] * grid_width for _ in range(grid_height)]

In [54]:
shape1 = [[1,2,3]]
grid = fill_array(val=0,grid_width=1,grid_height=3)


print_mas(grid)

print(can_place(grid, shape1,0,0))
print_mas(shape1)


shape1 = rotate_shape(shape1)

print(can_place(grid, shape1,0,0))
print_mas(shape1)


[0]
[0]
[0]

Текущая ячейка: 0,0
Текущая ячейка: 0,1
Текущая ячейка: 0,2
True
[1, 2, 3]

Текущая ячейка: 0,0
Текущая ячейка: 1,0
Текущая ячейка: 2,0
True
[1]
[2]
[3]



In [56]:
import os
import json

def create_item_id_mapping(bags_dir, items_dir, output_file):
    all_files = []

    # Получаем список файлов из обеих папок
    for folder in [bags_dir, items_dir]:
        for filename in os.listdir(folder):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.webp')):  # допустимые форматы
                name = os.path.splitext(filename)[0]  # без расширения
                all_files.append(name)

    # Удалим дубликаты и отсортируем (по желанию)
    unique_names = set(all_files)

    # Создаём словарь: имя -> id
    name_to_id = {name: idx for idx, name in enumerate(unique_names)}

    # Сохраняем в JSON
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(name_to_id, f, ensure_ascii=False, indent=4)

    print(f'Готово! Словарь сохранён в {output_file}')

# Пример использования
create_item_id_mapping('resized_items/bags', 'resized_items/items', 'item_id_map.json')


Готово! Словарь сохранён в item_id_map.json


In [57]:
from PIL import Image, ImageDraw, ImageFilter
import random

def generate_stylized_cell(size=160):
    base_color = (180, 150, 110)  # светло-коричневый
    highlight = (200, 170, 130)
    shadow = (100, 80, 60)

    img = Image.new("RGBA", (size, size), base_color)
    draw = ImageDraw.Draw(img)

    # Мягкая внутренняя заливка
    inner_margin = 10
    draw.rounded_rectangle(
        [inner_margin, inner_margin, size-inner_margin, size-inner_margin],
        radius=20,
        fill=highlight
    )

    # Тень по краям (градиентное размытие)
    shadow_layer = Image.new("RGBA", (size, size), (0, 0, 0, 0))
    shadow_draw = ImageDraw.Draw(shadow_layer)
    shadow_draw.rounded_rectangle(
        [0, 0, size, size],
        radius=30,
        fill=shadow + (60,)  # прозрачная тень
    )
    shadow_layer = shadow_layer.filter(ImageFilter.GaussianBlur(6))
    img = Image.alpha_composite(shadow_layer, img)

    # Лёгкий "шум" или потертость
    noise = Image.effect_noise((size, size), 10)
    noise = noise.convert("L").point(lambda x: x // 3)
    noise = noise.convert("RGBA")
    img = Image.alpha_composite(img, noise)

    return img

# Пример использования
cell_texture = generate_stylized_cell()
cell_texture.save("stylized_cell.png")


Окно с реалтайм обработкой 

In [8]:
import cv2
import json
import random

# Загрузка классов
with open('item_id_map.json', 'r', encoding='utf-8') as f:
    name_to_id = json.load(f)

id_to_name = {v: k for k, v in name_to_id.items()}

# Генерируем случайные цвета для каждого класса
class_colors = {
    class_id: (random.randint(50, 255), random.randint(50, 255), random.randint(50, 255))
    for class_id in id_to_name
}

# Загрузим фото
image = cv2.imread('dataset/images/train/1.png')
height, width = image.shape[:2]

# Прочитаем аннотации
with open('dataset/labels/train/1.txt', 'r') as f:
    lines = f.readlines()

for line in lines:
    class_id, x_center, y_center, w, h = map(float, line.strip().split())
    x_center *= width
    y_center *= height
    w *= width
    h *= height

    x1 = int(x_center - w / 2)
    y1 = int(y_center - h / 2)
    x2 = int(x_center + w / 2)
    y2 = int(y_center + h / 2)

    color = class_colors[int(class_id)]
    label = id_to_name[int(class_id)]

    # Нарисуем прямоугольник
    cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)

    # Нарисуем подпись с названием класса
    cv2.putText(image, label, (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

# Покажем изображение
cv2.imshow('Image with boxes', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Разделение train images и labels на train и val

In [1]:
import os
import shutil
import random
from pathlib import Path
import yaml
import json

# Один раз загружаем словарь из файла
with open('item_id_map.json', 'r', encoding='utf-8') as f:
    name_to_id = json.load(f)

# Параметры
dataset_dir = Path("dataset")
images_dir = dataset_dir / "images" / "train_full"
labels_dir = dataset_dir / "labels" / "train_full"

train_images_dir = dataset_dir / "images" / "train"
val_images_dir = dataset_dir / "images" / "val"
train_labels_dir = dataset_dir / "labels" / "train"
val_labels_dir = dataset_dir / "labels" / "val"

val_ratio = 0.2  # 20% для валидации

# Создаём папки, если их нет
for d in [train_images_dir, val_images_dir, train_labels_dir, val_labels_dir]:
    d.mkdir(parents=True, exist_ok=True)

# Получаем список всех файлов изображений
all_images = sorted(images_dir.glob("*.png"))  # подстрой под формат, если другой

# Перемешиваем для случайного разбиения
random.shuffle(all_images)

num_val = int(len(all_images) * val_ratio)
val_images = all_images[:num_val]
train_images = all_images[num_val:]

def copy_files(file_list, images_dst, labels_dst):
    for img_path in file_list:
        label_path = labels_dir / (img_path.stem + ".txt")

        shutil.copy(img_path, images_dst / img_path.name)
        if label_path.exists():
            shutil.copy(label_path, labels_dst / label_path.name)
        else:
            print(f"Внимание: нет аннотации для {img_path.name}")

# Копируем файлы
copy_files(train_images, train_images_dir, train_labels_dir)
copy_files(val_images, val_images_dir, val_labels_dir)

print(f"Train: {len(train_images)} изображений")
print(f"Val: {len(val_images)} изображений")

# Генерация data.yaml для YOLOv8
data_yaml = {
    'path': str(dataset_dir.resolve()),  # корневая папка dataset
    'train': str(train_images_dir.relative_to(dataset_dir)),
    'val': str(val_images_dir.relative_to(dataset_dir)),
    'nc': len(name_to_id),  # количество классов, у тебя должно быть определено name_to_id
    'names': [None] * len(name_to_id)
}

# Заполним имена классов в порядке id (предполагается, что name_to_id — dict {name: id})
for name, idx in name_to_id.items():
    data_yaml['names'][idx] = name

yaml_path = dataset_dir / "data.yaml"
with open(yaml_path, "w") as f:
    yaml.dump(data_yaml, f, sort_keys=False)

print(f"Сгенерирован файл {yaml_path}")


Train: 0 изображений
Val: 0 изображений
Сгенерирован файл dataset\data.yaml
